In [1]:
from __future__ import print_function
import pandas as pd
import numpy as np

import matplotlib.pyplot as plt
import seaborn as sns
import pickle

import xgboost as xgb
from sklearn import preprocessing
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from xgboost import XGBClassifier
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import cross_val_score
from sklearn.metrics import classification_report,confusion_matrix
from sklearn import metrics
from sklearn.externals import joblib
import sklearn
import sklearn.datasets
import sklearn.ensemble
import lime
import lime.lime_tabular
import shap

In [37]:
df001 = pd.read_csv("C:/Users/t158532/Downloads/POC_5_20_v3.csv",low_memory=False)
df002 = pd.read_csv("C:/Users/t158532/Downloads/Unknown_without_weight.csv",low_memory=False)

df00 = [df001,df002]
df = pd.concat(df00)


In [38]:
dfOrig=df

df.fillna(0, inplace=True)
# Convert all object data type to string to do encoding
for col in df.columns.values:
    df[col]=df[col].fillna('0').astype('str')
    

dfEditCode = df['b.inrule_edit_code']

#drop unnecssary features
df = df.drop('b.inrule_edit_code', axis=1)
df = df.drop('inrule_jobname', axis=1)
df = df.drop('a.hit_bypass_edits_code', axis=1)
df = df.drop('a.hit_ci_qhit_ind', axis=1)
df = df.drop('a.hit_ci_qhit_rule', axis=1)
df = df.drop('a.hit_ciseg_causing_hit', axis=1)
df = df.drop('a.hit_dis_vs_rej_fl', axis=1)
df = df.drop('b.inrule_record_num', axis=1)

In [17]:
df.count()

a.hit_ci_carrier_code               124915
hit_procedure_code_5                124915
a.hit_ci_carrier_office_code        124915
a.hit_ci_orig_source_code_create    124915
a.hit_pcf_orig_claim_type           124915
a.hit_ci_hmo_coverage_flag          124915
a.hit_claim_type                    124915
hit_ci_group_number                 124915
dtype: int64

In [28]:
encoder_a_hit_ci_carrier_code= preprocessing.LabelEncoder()
df["a.hit_ci_carrier_code"]=encoder_a_hit_ci_carrier_code.fit_transform(df["a.hit_ci_carrier_code"])

encoder_a_hit_procedure_code_5= preprocessing.LabelEncoder()
df["hit_procedure_code_5"]=encoder_a_hit_procedure_code_5.fit_transform(df["hit_procedure_code_5"])

encoder_a_hit_ci_carrier_office_code= preprocessing.LabelEncoder()
df["a.hit_ci_carrier_office_code"]=encoder_a_hit_ci_carrier_office_code.fit_transform(df["a.hit_ci_carrier_office_code"])

encoder_a_hit_ci_orig_source_code_create= preprocessing.LabelEncoder()
df["a.hit_ci_orig_source_code_create"]=encoder_a_hit_ci_orig_source_code_create.fit_transform(df["a.hit_ci_orig_source_code_create"])

encoder_a_hit_pcf_orig_claim_type= preprocessing.LabelEncoder()
df["a.hit_pcf_orig_claim_type"]=encoder_a_hit_pcf_orig_claim_type.fit_transform(df["a.hit_pcf_orig_claim_type"])

encoder_a_hit_ci_hmo_coverage_flag= preprocessing.LabelEncoder()
df["a.hit_ci_hmo_coverage_flag"]=encoder_a_hit_ci_hmo_coverage_flag.fit_transform(df["a.hit_ci_hmo_coverage_flag"])

encoder_a_hit_claim_type= preprocessing.LabelEncoder()
df["a.hit_claim_type"]=encoder_a_hit_claim_type.fit_transform(df["a.hit_claim_type"])

encoder_a_hit_ci_group_number= preprocessing.LabelEncoder()
df["hit_ci_group_number"]=encoder_a_hit_ci_group_number.fit_transform(df["hit_ci_group_number"])

In [ ]:
df.columns

In [ ]:
array = df.values
X = array[:,0:8]

In [ ]:
X.shape

In [29]:
X_train, X_test, y_train, y_test = train_test_split(df, dfEditCode, test_size=0.30, random_state=0)

In [ ]:
y_test

In [ ]:
model = XGBClassifier()

parameters = {'nthread':[4], #when use hyperthread, xgboost may become slower
              'objective':['multi:softmax'],
              'learning_rate': [0.1], #set to control the weighting of new trees added to the model.
              'max_depth': [3],
              'min_child_weight': [1],
              'silent': [1],
              'num_class':[22],
              'subsample': [0.5],
              'colsample_bytree': [0.5], # fraction of columns to be randomly samples for each tree.
              'n_estimators': [1000], #number of trees, change it to 1000 for better results
              'missing':[-999],
              'seed': [1337]}


clf = GridSearchCV(estimator=model, param_grid=parameters, cv= 10)

clf.fit(X_train, y_train)


clf.best_score_
clf.score(X_test,y_test)

In [ ]:
#trust your CV!
clf.best_params_

In [ ]:
clf.best_score_
clf.score(X_test,y_test)

In [30]:
model = XGBClassifier(
    learning_rate=0.1,
    n_estimators=1000,
    max_depth=10,
    min_child_weight=11,
    gamma=0,
    subsample=0.8,
    colsample_bytree=0.8,
    num_class=100,
    objective='multi:softmax',
    nthread=4,
    scale_pos_weight=1,
    seed=1337
)

eval_set = [(X_test, y_test)]
model.fit(X_train, y_train, eval_metric="mlogloss", early_stopping_rounds=75, eval_set=eval_set, verbose=True)

[0]	validation_0-mlogloss:0.959745
Will train until validation_0-mlogloss hasn't improved in 75 rounds.
[1]	validation_0-mlogloss:0.849027
[2]	validation_0-mlogloss:0.760192
[3]	validation_0-mlogloss:0.68334
[4]	validation_0-mlogloss:0.616711
[5]	validation_0-mlogloss:0.558212
[6]	validation_0-mlogloss:0.506574
[7]	validation_0-mlogloss:0.45961
[8]	validation_0-mlogloss:0.417952
[9]	validation_0-mlogloss:0.38038
[10]	validation_0-mlogloss:0.346633
[11]	validation_0-mlogloss:0.316848
[12]	validation_0-mlogloss:0.289567
[13]	validation_0-mlogloss:0.265032
[14]	validation_0-mlogloss:0.243103
[15]	validation_0-mlogloss:0.22366
[16]	validation_0-mlogloss:0.205925
[17]	validation_0-mlogloss:0.189325
[18]	validation_0-mlogloss:0.1743
[19]	validation_0-mlogloss:0.161064
[20]	validation_0-mlogloss:0.148613
[21]	validation_0-mlogloss:0.137237
[22]	validation_0-mlogloss:0.126982
[23]	validation_0-mlogloss:0.117777
[24]	validation_0-mlogloss:0.109409
[25]	validation_0-mlogloss:0.101674
[26]	valida

[224]	validation_0-mlogloss:0.020685
[225]	validation_0-mlogloss:0.020676
[226]	validation_0-mlogloss:0.02067
[227]	validation_0-mlogloss:0.020667
[228]	validation_0-mlogloss:0.020665
[229]	validation_0-mlogloss:0.020669
[230]	validation_0-mlogloss:0.020663
[231]	validation_0-mlogloss:0.020666
[232]	validation_0-mlogloss:0.020669
[233]	validation_0-mlogloss:0.020668
[234]	validation_0-mlogloss:0.020667
[235]	validation_0-mlogloss:0.020667
[236]	validation_0-mlogloss:0.020668
[237]	validation_0-mlogloss:0.020658
[238]	validation_0-mlogloss:0.020663
[239]	validation_0-mlogloss:0.020668
[240]	validation_0-mlogloss:0.020666
[241]	validation_0-mlogloss:0.020662
[242]	validation_0-mlogloss:0.020667
[243]	validation_0-mlogloss:0.020667
[244]	validation_0-mlogloss:0.020668
[245]	validation_0-mlogloss:0.020663
[246]	validation_0-mlogloss:0.020662
[247]	validation_0-mlogloss:0.020657
[248]	validation_0-mlogloss:0.020652
[249]	validation_0-mlogloss:0.020649
[250]	validation_0-mlogloss:0.020652
[2

[447]	validation_0-mlogloss:0.020471
[448]	validation_0-mlogloss:0.020475
[449]	validation_0-mlogloss:0.020469
[450]	validation_0-mlogloss:0.020465
[451]	validation_0-mlogloss:0.02047
[452]	validation_0-mlogloss:0.020473
[453]	validation_0-mlogloss:0.020474
[454]	validation_0-mlogloss:0.020475
[455]	validation_0-mlogloss:0.020486
[456]	validation_0-mlogloss:0.020489
[457]	validation_0-mlogloss:0.020488
[458]	validation_0-mlogloss:0.020485
[459]	validation_0-mlogloss:0.02049
[460]	validation_0-mlogloss:0.020488
[461]	validation_0-mlogloss:0.020494
[462]	validation_0-mlogloss:0.020491
[463]	validation_0-mlogloss:0.020488
[464]	validation_0-mlogloss:0.020491
[465]	validation_0-mlogloss:0.020486
[466]	validation_0-mlogloss:0.020483
[467]	validation_0-mlogloss:0.020477
[468]	validation_0-mlogloss:0.02047
[469]	validation_0-mlogloss:0.020467
[470]	validation_0-mlogloss:0.020459
[471]	validation_0-mlogloss:0.020453
[472]	validation_0-mlogloss:0.020452
[473]	validation_0-mlogloss:0.020452
[474

[670]	validation_0-mlogloss:0.020373
[671]	validation_0-mlogloss:0.020371
[672]	validation_0-mlogloss:0.020376
[673]	validation_0-mlogloss:0.020372
[674]	validation_0-mlogloss:0.020374
[675]	validation_0-mlogloss:0.020375
[676]	validation_0-mlogloss:0.020373
[677]	validation_0-mlogloss:0.020373
[678]	validation_0-mlogloss:0.020374
[679]	validation_0-mlogloss:0.020371
[680]	validation_0-mlogloss:0.020372
[681]	validation_0-mlogloss:0.020374
[682]	validation_0-mlogloss:0.020374
[683]	validation_0-mlogloss:0.020378
[684]	validation_0-mlogloss:0.020377
[685]	validation_0-mlogloss:0.02038
[686]	validation_0-mlogloss:0.020386
[687]	validation_0-mlogloss:0.020391
[688]	validation_0-mlogloss:0.020389
[689]	validation_0-mlogloss:0.020393
[690]	validation_0-mlogloss:0.020394
[691]	validation_0-mlogloss:0.020398
[692]	validation_0-mlogloss:0.020401
[693]	validation_0-mlogloss:0.020402
[694]	validation_0-mlogloss:0.020402
[695]	validation_0-mlogloss:0.020399
[696]	validation_0-mlogloss:0.020403
[6

[893]	validation_0-mlogloss:0.020342
[894]	validation_0-mlogloss:0.020343
[895]	validation_0-mlogloss:0.020346
[896]	validation_0-mlogloss:0.020347
[897]	validation_0-mlogloss:0.020347
[898]	validation_0-mlogloss:0.020345
[899]	validation_0-mlogloss:0.020342
[900]	validation_0-mlogloss:0.020341
[901]	validation_0-mlogloss:0.02034
[902]	validation_0-mlogloss:0.020336
[903]	validation_0-mlogloss:0.020338
[904]	validation_0-mlogloss:0.020336
[905]	validation_0-mlogloss:0.020335
[906]	validation_0-mlogloss:0.020335
[907]	validation_0-mlogloss:0.020337
[908]	validation_0-mlogloss:0.020336
[909]	validation_0-mlogloss:0.020333
[910]	validation_0-mlogloss:0.020334
[911]	validation_0-mlogloss:0.020329
[912]	validation_0-mlogloss:0.020332
[913]	validation_0-mlogloss:0.020332
[914]	validation_0-mlogloss:0.020333
[915]	validation_0-mlogloss:0.020335
[916]	validation_0-mlogloss:0.020333
[917]	validation_0-mlogloss:0.020335
[918]	validation_0-mlogloss:0.020336
[919]	validation_0-mlogloss:0.020339
[9

XGBClassifier(base_score=0.5, booster='gbtree', colsample_bylevel=1,
       colsample_bytree=0.8, gamma=0, learning_rate=0.1, max_delta_step=0,
       max_depth=10, min_child_weight=11, missing=None, n_estimators=1000,
       n_jobs=1, nthread=4, num_class=100, objective='multi:softprob',
       random_state=0, reg_alpha=0, reg_lambda=1, scale_pos_weight=1,
       seed=1337, silent=True, subsample=0.8)

In [31]:
y_pred = model.predict(X_test)
print(confusion_matrix(y_test,y_pred))
print(classification_report(y_test,y_pred))
cls_report=classification_report(y_test,y_pred)

[[    0     0     0 ...     0     0     0]
 [    0    24     0 ...     0     0     0]
 [    0     0   214 ...     0     0     4]
 ...
 [    0     0     0 ...    12    14     0]
 [    0     0     0 ...    17     5     0]
 [    0     0     0 ...     0     0 20756]]


C:\ProgramData\Anaconda3\lib\site-packages\sklearn\metrics\classification.py:1143: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)
C:\ProgramData\Anaconda3\lib\site-packages\sklearn\metrics\classification.py:1143: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)
C:\ProgramData\Anaconda3\lib\site-packages\sklearn\metrics\classification.py:1143: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


               precision    recall  f1-score   support

90000_999_015       0.00      0.00      0.00         1
90000_999_031       1.00      1.00      1.00        24
90000_999_032       1.00      0.98      0.99       218
90000_999_111       1.00      1.00      1.00         6
90000_999_112       0.00      0.00      0.00         4
90000_999_113       0.83      1.00      0.91        49
90000_999_119       0.00      0.00      0.00         6
90000_999_146       1.00      1.00      1.00      1150
90000_999_164       0.00      0.00      0.00         1
90000_999_165       0.98      1.00      0.99        50
90000_999_170       1.00      1.00      1.00        16
90000_999_174       0.00      0.00      0.00         1
90000_999_175       0.98      1.00      0.99        56
90000_999_177       1.00      1.00      1.00        12
90000_999_180       0.00      0.00      0.00         7
90000_999_185       1.00      1.00      1.00        12
90000_999_188       0.00      0.00      0.00         7
90000_999

C:\ProgramData\Anaconda3\lib\site-packages\sklearn\metrics\classification.py:1143: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)
C:\ProgramData\Anaconda3\lib\site-packages\sklearn\metrics\classification.py:1143: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)
C:\ProgramData\Anaconda3\lib\site-packages\sklearn\metrics\classification.py:1143: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


In [32]:
print (metrics.accuracy_score(y_test, y_pred))

0.9938890964401985


In [ ]:
scores = cross_val_score(model, X_test, y_test, cv=10)
print(scores)
#print (metrics.accuracy_score(y_test, y_pred))

In [35]:
from  sklearn.metrics import precision_recall_fscore_support
from sklearn.metrics import roc_curve, auc
from sklearn.preprocessing import LabelBinarizer
from matplotlib import pyplot
from matplotlib import pyplot as plt
def class_report(y_true, y_pred, y_score=None, average='micro'):
    if y_true.shape != y_pred.shape:
        print("Error! y_true %s is not the same shape as y_pred %s" % (
              y_true.shape,
              y_pred.shape)
        )
        return

    lb = LabelBinarizer()

    if len(y_true.shape) == 1:
        lb.fit(y_true)

    #Value counts of predictions
    labels, cnt = np.unique(
        y_pred,
        return_counts=True)
    n_classes = len(labels)
    pred_cnt = pd.Series(cnt, index=labels)

    metrics_summary = precision_recall_fscore_support(
            y_true=y_true,
            y_pred=y_pred,
            labels=labels)

    avg = list(precision_recall_fscore_support(
            y_true=y_true, 
            y_pred=y_pred,
            average='weighted'))

    metrics_sum_index = ['precision', 'recall', 'f1-score', 'support']
    class_report_df = pd.DataFrame(
        list(metrics_summary),
        index=metrics_sum_index,
        columns=labels)

    support = class_report_df.loc['support']
    total = support.sum() 
    class_report_df['avg / total'] = avg[:-1] + [total]

    class_report_df = class_report_df.T
    class_report_df['pred'] = pred_cnt
    class_report_df['pred'].iloc[-1] = total

    if not (y_score is None):
        fpr = dict()
        tpr = dict()
        roc_auc = dict()
        for label_it, label in enumerate(labels):
            fpr[label], tpr[label], _ = roc_curve(
                (y_true == label).astype(int), 
                y_score[:, label_it])
            roc_auc[label] = auc(fpr[label], tpr[label])
        if average == 'micro':
            if n_classes <= 2:
                fpr["avg / total"], tpr["avg / total"], _ = roc_curve(
                    lb.transform(y_true).ravel(), 
                    y_score[:, 1].ravel())
            else:
                fpr["avg / total"], tpr["avg / total"], _ = roc_curve(
                        lb.transform(y_true).ravel(), 
                        y_score.ravel())

            roc_auc["avg / total"] = auc(
                fpr["avg / total"], 
                tpr["avg / total"])

        elif average == 'macro':
            # First aggregate all false positive rates
            all_fpr = np.unique(np.concatenate([
                fpr[i] for i in labels]
            ))

            # Then interpolate all ROC curves at this points
            mean_tpr = np.zeros_like(all_fpr)
            for i in labels:
                mean_tpr += interp(all_fpr, fpr[i], tpr[i])

            # Finally average it and compute AUC
            mean_tpr /= n_classes

            fpr["macro"] = all_fpr
            tpr["macro"] = mean_tpr

            roc_auc["avg / total"] = auc(fpr["macro"], tpr["macro"])

        class_report_df['AUC'] = pd.Series(roc_auc)

    return class_report_df

report_with_auc = class_report(
    y_true=y_test, 
    y_pred=model.predict(X_test))
    #y_score=model.predict_proba(X_test))

print(report_with_auc)

C:\ProgramData\Anaconda3\lib\site-packages\sklearn\metrics\classification.py:1143: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


               precision    recall  f1-score  support     pred
90000_999_031   1.000000  1.000000  1.000000     24.0     24.0
90000_999_032   1.000000  0.981651  0.990741    218.0    214.0
90000_999_111   1.000000  1.000000  1.000000      6.0      6.0
90000_999_113   0.830508  1.000000  0.907407     49.0     59.0
90000_999_146   1.000000  1.000000  1.000000   1150.0   1150.0
90000_999_165   0.980392  1.000000  0.990099     50.0     51.0
90000_999_170   1.000000  1.000000  1.000000     16.0     16.0
90000_999_175   0.982456  1.000000  0.991150     56.0     57.0
90000_999_177   1.000000  1.000000  1.000000     12.0     12.0
90000_999_185   1.000000  1.000000  1.000000     12.0     12.0
90000_999_188   0.000000  0.000000  0.000000      7.0      7.0
90000_999_189   0.758621  1.000000  0.862745     22.0     29.0
90000_999_194   0.000000  0.000000  0.000000     14.0      3.0
90000_999_196   1.000000  1.000000  1.000000    201.0    201.0
90000_999_197   1.000000  1.000000  1.000000     29.0  

In [36]:
#100 of data

y_pred = model.predict(df)
print(confusion_matrix(dfEditCode,y_pred))
print(classification_report(dfEditCode,y_pred))
cls_report=classification_report(dfEditCode,y_pred)

[[    0     0     0 ...     0     0     0]
 [    0    61     0 ...     0     0     0]
 [    0     0   719 ...     0     0     6]
 ...
 [    0     0     0 ...    46    32     0]
 [    0     0     0 ...    46    32     0]
 [    0     0     0 ...     0     0 68821]]


C:\ProgramData\Anaconda3\lib\site-packages\sklearn\metrics\classification.py:1143: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)
C:\ProgramData\Anaconda3\lib\site-packages\sklearn\metrics\classification.py:1143: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)
C:\ProgramData\Anaconda3\lib\site-packages\sklearn\metrics\classification.py:1143: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


               precision    recall  f1-score   support

90000_999_015       0.00      0.00      0.00         5
90000_999_031       1.00      1.00      1.00        61
90000_999_032       1.00      0.99      1.00       725
90000_999_111       1.00      1.00      1.00        20
90000_999_112       0.00      0.00      0.00        14
90000_999_113       0.88      1.00      0.93       170
90000_999_119       0.00      0.00      0.00        10
90000_999_146       1.00      1.00      1.00      3881
90000_999_164       0.00      0.00      0.00         7
90000_999_165       0.96      0.99      0.98       182
90000_999_170       1.00      1.00      1.00        59
90000_999_174       0.00      0.00      0.00         3
90000_999_175       0.99      1.00      0.99       218
90000_999_176       0.00      0.00      0.00         1
90000_999_177       0.97      1.00      0.99        39
90000_999_180       0.00      0.00      0.00        16
90000_999_185       0.88      1.00      0.93        21
90000_999

C:\ProgramData\Anaconda3\lib\site-packages\sklearn\metrics\classification.py:1143: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)
C:\ProgramData\Anaconda3\lib\site-packages\sklearn\metrics\classification.py:1143: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)
C:\ProgramData\Anaconda3\lib\site-packages\sklearn\metrics\classification.py:1143: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


In [ ]:
y_predproba = model.predict_proba(X)
print (y_predproba)

In [ ]:
dataset=pd.DataFrame(model.predict_proba(X), columns=model.classes_)

writer = pd.ExcelWriter('proba.xlsx')
dataset.to_excel(writer,'Sheet1')
writer.save()


In [ ]:
dfTest=pd.DataFrame(X_test)

In [ ]:
dfTest[:2]

In [ ]:
X_test.shape

In [ ]:
dfprdcCode=encoder_a_hit_procedure_code_5.inverse_transform(df["hit_procedure_code_5"])
dfoffCode=encoder_a_hit_ci_carrier_office_code.inverse_transform(df["a.hit_ci_carrier_office_code"])

In [ ]:
dfOrig.iloc[20000]
  
# making boolean series for a team name 
filter = dfOrig["b.inrule_edit_code"]=="90005_999_193"
  
# filtering data 
df193=dfOrig.where(filter, inplace = True) 
  
# display 
df193

In [ ]:
X_test_explain=X[:1000]

In [ ]:
X_test_explain[:13]

In [ ]:
explainer = shap.TreeExplainer(model)
shap_values = explainer.shap_values(X_test_explain)

In [ ]:
df.shape

In [ ]:
X_test_explain.shape

In [ ]:
#shap_values[1,:]
shapval=shap_values[0][:1000]
shapval[10]

In [ ]:
X_test_explain[13]

In [ ]:
shap.initjs()
shap.force_plot(explainer.expected_value[0], shapval[0][:], X_test_explain[13][:].astype(str))

In [ ]:
result = model.predict(X)
print(result)

In [ ]:
resultDf=pd.DataFrame(result)
X_test_pd =pd.DataFrame(pd.DataFrame(X))

In [ ]:
editCodeDf=pd.DataFrame(dfEditCode)

In [ ]:
editCodeDf.shape

In [ ]:
#Save results into spreasheet for review
dfOrig.reset_index(inplace=True, drop=True)

finals=pd.concat([dfOrig, resultDf],axis=1, sort=False)

writer = pd.ExcelWriter('POC_Explainer2.xlsx')
finals.to_excel(writer,'Sheet1')
writer.save()

In [ ]:
final=pd.concat([X_test_pd, resultDf],axis=1)

In [ ]:
final[:10]

In [ ]:
writer = pd.ExcelWriter('XgBoost_LIME1.xlsx')
final.to_excel(writer,'Sheet1')
writer.save()

In [ ]:
shap_values[0][:1]

In [ ]:
X_test_explain[13][:]

In [ ]:
shap.force_plot(explainer.expected_value[0], shapval[13][:], X_test_explain[13][:].astype(str))

In [ ]:
X_test_explain[14][:]

In [ ]:
shap.force_plot(explainer.expected_value[0], shapval[14][:], X_test_explain[14][:].astype(str))

In [ ]:
shap.summary_plot(shap_values[:1], X_test_explain)

In [ ]:
shap.summary_plot(shap_values[0:2], X_test_explain)

In [ ]:
shap.summary_plot(shap_values[0], X_test_explain)

In [ ]:
shap.summary_plot(shap_values[0], X_test_explain)

In [ ]:
shap.summary_plot(shapval, X_test_explain, plot_type="bar")

In [ ]:
shap.summary_plot(shap_values, X_test_explain)

In [ ]:
cat_columns=[]
for col in df.columns.values:
    cat_columns.append(col)

train_cat_features = df[cat_columns]
feature_names =train_cat_features

class_names=[]
class_names=['10100',
'10120',      
'21313',           
'25020',       
'25021',       
'31318',       
'33330',       
'33337',       
'55555',       
'66662',       
'90000_999_503',
'90000_999_598',       
'90000_999_826',      
'90002_999_477',       
'90003_999_382',       
'90004_564_004',       
'90004_999_123',       
'90004_999_512',       
'90005_999_169',       
'90005_999_193',       
'90005_999_475',       
'Unkown']


predict_fn_xgb = lambda x: model.predict_proba(x).astype(float)

explainer = lime.lime_tabular.LimeTabularExplainer(X_test_explain ,feature_names=feature_names,mode='classification',
                                                   categorical_features=cat_columns,
                                                   class_names=class_names)

In [ ]:
model.predict_proba(X_test_explain)[0:10, 1]

In [ ]:
X_test_pd.iloc[13]

In [ ]:
#dfOrig.iloc[25000]
X_test_explain[0:10]

In [ ]:
#Incorrect Classification
observation_1 = 14

# Get the explanation for RandomForest
exp = explainer.explain_instance(X_test_explain[observation_1], predict_fn_xgb, num_features=13,labels=(0,))
exp.show_in_notebook(show_all=False)

In [ ]:
dfOrig.iloc[13]

In [ ]:
observation_1 = 13

# Get the explanation for RandomForest
exp = explainer.explain_instance(X_test_explain[observation_1], predict_fn_xgb, num_features=13,labels=(0,))
exp.show_in_notebook(show_all=False)

In [ ]:
#print(model.predict_proba([X_test[0]]))

#print('Probability(90004_999_123) =', model.predict_proba([X_test[observation_1]])[0,1])
#print('True class: %s' % class_names[X_test[observation_1]])

#idx = 13
#exp = explainer.explain_instance(X_test1[idx], model.predict_proba, num_features=13, labels=[0,4])
#print('Document id: %d' % idx)
#print('Predicted class =', class_names[model.predict(test_vectors[idx]).reshape(1,-1)[0,0]])
#print('True class: %s' % class_names[newsgroups_test.target[idx]])

In [ ]:
exp.as_list()

In [ ]:
exp.as_pyplot_figure()
from matplotlib import pyplot as plt
plt.tight_layout()

In [ ]:
exp1 = explainer.explain_instance(X_test_explain[13],predict_fn_xgb, num_features=13,labels=[4]) 
exp1.as_pyplot_figure()
plt.tight_layout()


exp1 =  explainer.explain_instance(X_test1[:13], predict_fn_xgb, num_features=13,labels=[0,4])
exp1.as_pyplot_figure()
plt.tight_layout()

In [ ]:
dfE=pd.DataFrame(dfEditCode)
class_names=[]
for col in dfE.columns.values:
    class_names=dfE[col]

In [ ]:
from lime.lime_text import LimeTextExplainer
explainer = LimeTextExplainer(class_names=class_names)

In [ ]:
exp1 =  explainer.explain_instance(X_test_explain[observation_1], predict_fn_xgb, num_features=13,labels=[0, 4])

In [ ]:
print(exp.available_labels())

In [ ]:
exp.show_in_notebook(show_all=False)